#Set up

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
%pip install ultralytics
!pip install roboflow

In [ ]:
#from roboflow import Roboflow
#rf = Roboflow(api_key="uK1Gw9yHOv6V9YSi3Fj4")
#project = rf.workspace("joseph-nelson").project("pistols")
#version = project.version(1)
#dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.31, to fix: `pip install ultralytics==8.0.196`


#Making dataset

Make directories

In [ ]:
# !mkdir -p '/content/drive/MyDrive/DL - GP/dataset/images/train'
# !mkdir -p '/content/drive/MyDrive/DL - GP/dataset/images/val'
# !mkdir -p '/content/drive/MyDrive/DL - GP/dataset/images/test'
# !mkdir -p '/content/drive/MyDrive/DL - GP/dataset/labels/train'
# !mkdir -p '/content/drive/MyDrive/DL - GP/dataset/labels/val'
# !mkdir -p '/content/drive/MyDrive/DL - GP/dataset/labels/test'

In [ ]:
# import os
# import shutil
# import random

# # Define paths
# images_dir = 'path/to/your/images'
# labels_dir = 'path/to/your/labels'
# output_dir = 'dataset'

# # Define output directories
# train_images_dir = os.path.join(output_dir, 'images/train')
# val_images_dir = os.path.join(output_dir, 'images/val')
# test_images_dir = os.path.join(output_dir, 'images/test')

# train_labels_dir = os.path.join(output_dir, 'labels/train')
# val_labels_dir = os.path.join(output_dir, 'labels/val')
# test_labels_dir = os.path.join(output_dir, 'labels/test')

# # Create output directories if they don't exist
# os.makedirs(train_images_dir, exist_ok=True)
# os.makedirs(val_images_dir, exist_ok=True)
# os.makedirs(test_images_dir, exist_ok=True)
# os.makedirs(train_labels_dir, exist_ok=True)
# os.makedirs(val_labels_dir, exist_ok=True)
# os.makedirs(test_labels_dir, exist_ok=True)

# # Get list of image files
# image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

# # Define split ratios
# train_ratio = 0.7
# val_ratio = 0.2
# test_ratio = 0.1

# # Calculate split indices
# train_end = int(train_ratio * len(image_files))
# val_end = train_end + int(val_ratio * len(image_files))

# # Split the data
# train_files = image_files[:train_end]
# val_files = image_files[train_end:val_end]
# test_files = image_files[val_end:]

# def move_files(files, src_dir, dest_img_dir, dest_lbl_dir):
#     for file in files:
#         image_path = os.path.join(src_dir, file)
#         label_path = os.path.join(labels_dir, file.replace('.jpg', '.txt').replace('.png', '.txt'))

#         if os.path.exists(label_path):
#             shutil.copy(image_path, dest_img_dir)
#             shutil.copy(label_path, dest_lbl_dir)
#         else:
#             print(f"Label for {file} not found!")

# # Move the files
# move_files(train_files, images_dir, train_images_dir, train_labels_dir)
# move_files(val_files, images_dir, val_images_dir, val_labels_dir)
# move_files(test_files, images_dir, test_images_dir, test_labels_dir)

# print("Data split completed!")


##Test to see if the labels and images correspond

#Train

In [5]:
from ultralytics import YOLO

In [6]:
model = YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 76.6MB/s]


In [7]:
data='/content/drive/MyDrive/DL - GP/dataset/data.yaml'

In [9]:
augmentation_params = {
    'hsv_h': 0.015,    # Adjust the hue of the image by a fraction
    'hsv_s': 0.7,      # Alters the saturation of the image by a fraction
    'hsv_v': 0.4,      # Modifies the value (brightness) of the image by a fraction
    'degrees': 10.0,   # Rotates the image randomly within the specified degree range
    'translate': 0.1,  # Translate the image by a fraction of the width/height
    'scale': 0.5,      # Scale the image by a fraction
    'shear': 2.0,      # Shear the image by a fraction
    'perspective': 0.0,  # Apply perspective transformation
    'flipud': 0.0,     # Flip the image vertically with probability
    'fliplr': 0.5,     # Flip the image horizontally with probability
    'mosaic': 1.0,     # Apply mosaic augmentation with probability
    'mixup': 0.0       # Apply mixup augmentation with probability
}

In [10]:
results = model.train(
    data=data,
    epochs=1,
    patience=7,
    # batch= \ can be non fixed.
    imgsz=640,
    save=True,
    device='cpu',  # zero for one gpu. 0,1 for multiple gpu
    pretrained=True,  # will this lock all and i have to add a new dense?
    optimizer='auto',
    verbose=True,
    seed=1,
    fraction=0.2,  # only to test it works. 1 for full training
    # freeze= dependent on what pretrained does
    # lr0= normal learning rate lrf= is fraction of initial learning rate at the end
    # weight_decay L2
    # dropout=
    val=True,
    plots=True,
    augment=True,
    **augmentation_params
)


Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/DL - GP/dataset/data.yaml, epochs=1, time=None, patience=7, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=1, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=0.2, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 15.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/DL - GP/dataset/labels/test.cache... 48 images, 0 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.817       3.04      2.058         40        640: 100%|██████████| 3/3 [00:45<00:00, 15.12s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [16:26<00:00, 18.62s/it]


                   all       1687       1962    0.00363      0.936      0.167     0.0874

1 epochs completed in 0.289 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [19:23<00:00, 21.94s/it]


                   all       1687       1962    0.00365      0.941      0.221      0.132
Speed: 7.3ms preprocess, 649.0ms inference, 0.0ms loss, 18.8ms postprocess per image
Results saved to runs/detect/train


#Test

In [13]:
metrics = model.val(plots=True) #add yaml for val data?, save_json, conf

Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)


val: Scanning /content/drive/MyDrive/DL - GP/dataset/labels/train.cache... 1687 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1687/1687 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 106/106 [18:04<00:00, 10.23s/it]


                   all       1687       1962    0.00365      0.941      0.221      0.132
Speed: 7.5ms preprocess, 583.9ms inference, 0.0ms loss, 18.8ms postprocess per image
Results saved to runs/detect/train3


In [15]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b036c61f130>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 